In [94]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.functional as f
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.autograd import Variable
from torchvision.utils import save_image

In [95]:
numEpochs = 100
lr =  1e-3
use_gpu = torch.cuda.is_available()
use_gpu

True

In [96]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [97]:
imgTransform = transforms.Compose([
    transforms.ToTensor(),
])

In [98]:
test = MNIST("../data/MNIST", transform=imgTransform, train=False, download=True)
train = MNIST("../data/MNIST", transform=imgTransform, train=True, download=True)

In [99]:
trainData = DataLoader(train, batch_size=128, shuffle=True)
testData = DataLoader(test, batch_size=128, shuffle=True)

In [100]:
# Autoencoder, used for feature extraction
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def encode(self, x):
        x = self.encoder(x)
        return x

In [101]:
featureExtractor = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(featureExtractor.parameters(), lr=lr,
                             weight_decay=1e-5)

In [102]:
for epoch in range(numEpochs):
    for data in trainData:
        img, _ = data
        img = Variable(img).cuda()

        output = featureExtractor(img)
        loss = criterion(output, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch, numEpochs, loss.data.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './dc_img/image_{}.png'.format(epoch))

epoch [0/100], loss:0.0549
epoch [1/100], loss:0.0463
epoch [2/100], loss:0.0407
epoch [3/100], loss:0.0416
epoch [4/100], loss:0.0372
epoch [5/100], loss:0.0363
epoch [6/100], loss:0.0345
epoch [7/100], loss:0.0341
epoch [8/100], loss:0.0325
epoch [9/100], loss:0.0311
epoch [10/100], loss:0.0307
epoch [11/100], loss:0.0294
epoch [12/100], loss:0.0293
epoch [13/100], loss:0.0307
epoch [14/100], loss:0.0284
epoch [15/100], loss:0.0276
epoch [16/100], loss:0.0271
epoch [17/100], loss:0.0274
epoch [18/100], loss:0.0275
epoch [19/100], loss:0.0270
epoch [20/100], loss:0.0262
epoch [21/100], loss:0.0267
epoch [22/100], loss:0.0289
epoch [23/100], loss:0.0253
epoch [24/100], loss:0.0253
epoch [25/100], loss:0.0250
epoch [26/100], loss:0.0253
epoch [27/100], loss:0.0270
epoch [28/100], loss:0.0271
epoch [29/100], loss:0.0277
epoch [30/100], loss:0.0267
epoch [31/100], loss:0.0273
epoch [32/100], loss:0.0286
epoch [33/100], loss:0.0279
epoch [34/100], loss:0.0271
epoch [35/100], loss:0.0262
ep

In [103]:
# validate
for data in testData:
    img, _ = data
    img = Variable(img).cuda()

    output = featureExtractor(img)
    loss = criterion(output, img)
    
print(loss.data.item())

0.021843060851097107


In [131]:
trainFeat = torch.empty(0,device='cuda:0',)
for data in trainData:
        img, _ = data
        img = Variable(img).cuda()

        output = featureExtractor.encode(img)
        trainFeat = torch.cat((trainFeat, output), 0)
        
print(trainFeat.shape)
trainFeat = testFeat.view(60000,32)
print(trainFeat.shape)

torch.Size([60000, 8, 2, 2])
torch.Size([60000, 32])


In [133]:
testFeat = torch.empty(0,device='cuda:0',)
for data in testData:
        img, _ = data
        img = Variable(img).cuda()

        output = featureExtractor.encode(img)
        testFeat = torch.cat((testFeat, output), 0)
print(testFeat.shape)
testFeat = testFeat.view(10000,32)
print(testFeat.shape)

torch.Size([10000, 8, 2, 2])
torch.Size([10000, 32])


In [144]:
torch.tensor.ToTenso r(test)

AttributeError: 'builtin_function_or_method' object has no attribute 'ToTensor'

In [139]:
# Root node in our Everfree Tree, only direction
class root(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.layer1 = nn.sequential(
            nn.Linear(32, 8),
            nn.ReLU(True))
        self.layer2 = nn.sequential(
            nn.Linear(8, 1),
            nn.sigmoid())
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [140]:
# leaf node, has two possible outputs, direction and error
class leaf(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.layer1 = nn.sequential(
            nn.Linear(32, 8),
            nn.ReLU(True))
        self.layer2 = nn.sequential(
            nn.Linear(8, 2),
            nn.sigmoid())
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x